
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_2887276/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 19:40:49.329587: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:40:49.361905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 19:40:49.361938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 19:40:49.363015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 19:40:49.368608: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:40:49.369553: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[70,35]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[70, 35], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.5179188350011827
Stopped at epoch:  32
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  1.2426278069997352
Stopped at epoch:  37
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.4985541660007584
Stopped at epoch:  16
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.21877273000063724
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.24884963700060325
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.2358651970007486
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.23969015999864496
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_2_8,0.808658,0.646157,0.273645,0.500017,23.075962,51.081917,93.150093,70.878792
model_1_2_7,0.809214,0.659701,0.273995,0.507468,23.008886,49.126648,93.105194,69.822403
model_1_2_4,0.813702,0.723651,0.370614,0.583078,22.467564,39.894577,80.714432,59.103859
model_1_2_6,0.816563,0.703152,0.377388,0.574910,22.122562,42.853886,79.845695,60.261784
model_1_2_3,0.818210,0.724463,0.416847,0.603197,21.923906,39.777374,74.785370,56.251740
model_1_2_0,0.819603,0.722174,0.461603,0.621015,21.755875,40.107864,69.045670,53.725719
model_1_2_2,0.820683,0.728679,0.440799,0.615667,21.625736,39.168743,71.713707,54.483952
model_1_2_5,0.820803,0.717786,0.405359,0.594706,21.611219,40.741394,76.258667,57.455467
model_1_2_1,0.820835,0.725509,0.456914,0.620818,21.607346,39.626369,69.647026,53.753746


DataFrame salvo em ../results/metrics_1_2
+++++++++++ [1_3] | 1 ++++++++++++++++++
Elapsed time:  0.19957810200139647
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 2 ++++++++++++++++++
Elapsed time:  0.19467716299914173
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 3 ++++++++++++++++++
Elapsed time:  0.2591026600002806
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 4 ++++++++++++++++++
Elapsed time:  0.17198949600060587
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_3] | 5 ++++++++++++++++++
Elapsed time:  0.1734533709986863
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 6 ++++++++++++++++++
Elapsed time:  0.1895608259983419
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_3] | 7 ++++++++++++++++++
Elapsed time:  0.167676905

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_4_0,0.803434,0.446911,0.941891,0.826020,23.705954,23.013678,8.001377,15.949062
model_1_4_1,0.809306,0.442793,0.941258,0.824583,22.997728,23.184990,8.088536,16.080776
model_1_4_2,0.814746,0.438477,0.940724,0.823168,22.341740,23.364597,8.162130,16.210495
model_1_4_3,0.819666,0.429545,0.939490,0.820150,21.748291,23.736267,8.332004,16.487209
model_1_4_4,0.823903,0.415035,0.937504,0.815259,21.237299,24.340019,8.605541,16.935560
model_1_4_5,0.827638,0.396092,0.934963,0.808912,20.786886,25.128225,8.955297,17.517435
model_1_4_18,0.828960,-0.350741,0.875866,0.587677,20.627481,56.203445,17.092813,37.798443
model_1_4_19,0.829289,-0.383478,0.870850,0.576265,20.587801,57.565598,17.783546,38.844635
model_1_4_6,0.830684,0.369701,0.932012,0.800484,20.419508,26.226309,9.361678,18.290009
model_1_4_17,0.831375,-0.266447,0.881736,0.612082,20.336195,52.696003,16.284542,35.561199


...

DataFrame salvo em ../results/metrics_3_7
+++++++++++ [3_8] | 1 ++++++++++++++++++
Elapsed time:  0.3313111180013948
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_8] | 2 ++++++++++++++++++
Elapsed time:  0.7866119910013367
Stopped at epoch:  12
1/1 [==============================] - 0s 19ms/step
+++++++++++ [3_8] | 3 ++++++++++++++++++
Elapsed time:  0.4462150319995999
Stopped at epoch:  7
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_8] | 4 ++++++++++++++++++
Elapsed time:  0.58878280999852
Stopped at epoch:  10
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_8] | 5 ++++++++++++++++++
Elapsed time:  0.454742105999685
Stopped at epoch:  10
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_8] | 6 ++++++++++++++++++
Elapsed time:  0.2607653110007959
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_8] | 7 ++++++++++++++++++
Elapsed time:  0.447546327000

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_8_9,0.910002,0.780314,0.919304,0.849914,10.853801,40.154373,10.251018,26.082205
model_3_8_0,0.911186,0.965168,0.920378,0.953215,10.710961,6.366540,10.114626,8.130344
model_3_8_10,0.911905,0.784741,0.918494,0.852100,10.624296,39.345123,10.353873,25.702187
model_3_8_1,0.927216,0.960696,0.927073,0.953028,8.777742,7.184061,9.264082,8.162895
model_3_8_2,0.930415,0.956295,0.929686,0.951476,8.392007,7.988436,8.932102,8.432513
model_3_8_3,0.936828,0.948108,0.928180,0.946399,7.618512,9.484887,9.123435,9.314793
model_3_8_4,0.943541,0.948310,0.925371,0.945546,6.809021,9.447934,9.480267,9.463149
model_3_8_5,0.944558,0.946914,0.925550,0.944830,6.686318,9.703136,9.457598,9.587586
model_3_8_6,0.945126,0.945144,0.924089,0.943342,6.617863,10.026659,9.643120,9.846168
model_3_8_7,0.948160,0.939078,0.924561,0.940126,6.251920,11.135425,9.583175,10.404956


DataFrame salvo em ../results/metrics_3_8
+++++++++++ [3_9] | 1 ++++++++++++++++++
Elapsed time:  0.46710134200111497
Stopped at epoch:  6
1/1 [==============================] - 0s 31ms/step
+++++++++++ [3_9] | 2 ++++++++++++++++++
Elapsed time:  0.8777949140003329
Stopped at epoch:  16
1/1 [==============================] - 0s 24ms/step
+++++++++++ [3_9] | 3 ++++++++++++++++++
Elapsed time:  0.48190500300006534
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [3_9] | 4 ++++++++++++++++++
Elapsed time:  0.3736100149999402
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
+++++++++++ [3_9] | 5 ++++++++++++++++++
Elapsed time:  0.33576580500084674
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_9] | 6 ++++++++++++++++++
Elapsed time:  0.4505992389986204
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_9] | 7 ++++++++++++++++++
Elapsed time:  0.39079084

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_9_24,0.906600,-0.615317,0.881976,0.653942,11.264028,49.208435,18.318512,34.671997
model_3_9_23,0.907068,-0.606744,0.884016,0.656809,11.207632,48.947254,18.001945,34.384769
model_3_9_22,0.907543,-0.598245,0.886168,0.659746,11.150313,48.688343,17.667917,34.090508
model_3_9_21,0.908052,-0.588504,0.888280,0.662854,11.088927,48.391617,17.340073,33.779129
model_3_9_20,0.908588,-0.578239,0.890490,0.666117,11.024340,48.078907,16.997055,33.452160
model_3_9_19,0.909198,-0.567271,0.893024,0.669730,10.950766,47.744785,16.603844,33.090225
model_3_9_18,0.909760,-0.556657,0.895389,0.673163,10.882944,47.421436,16.236641,32.746227
model_3_9_17,0.910332,-0.545924,0.897883,0.676708,10.813950,47.094471,15.849599,32.391018
model_3_9_16,0.910858,-0.536165,0.900425,0.680133,10.750526,46.797161,15.455088,32.047943
model_3_9_15,0.911442,-0.525121,0.903029,0.683809,10.680069,46.460720,15.050879,31.679623


DataFrame salvo em ../results/metrics_3_9
Testando combinacao4: Hidden Size=[70, 35], regularizer=0.1, learning_rate=0.1
+++++++++++ [4_0] | 1 ++++++++++++++++++
Elapsed time:  3.423762861000796
Stopped at epoch:  39
1/1 [==============================] - 0s 26ms/step
+++++++++++ [4_0] | 2 ++++++++++++++++++
Elapsed time:  0.5752667529995961
Stopped at epoch:  8
1/1 [==============================] - 0s 26ms/step
+++++++++++ [4_0] | 3 ++++++++++++++++++
Elapsed time:  1.6620172450002428
Stopped at epoch:  19
1/1 [==============================] - 0s 49ms/step
+++++++++++ [4_0] | 4 ++++++++++++++++++
Elapsed time:  1.8889606129996537
Stopped at epoch:  23
1/1 [==============================] - 0s 22ms/step
+++++++++++ [4_0] | 5 ++++++++++++++++++
Elapsed time:  0.741319877000933
Stopped at epoch:  8
1/1 [==============================] - 0s 23ms/step
+++++++++++ [4_0] | 6 ++++++++++++++++++
Elapsed time:  0.5113531240003795
Stopped at epoch:  6
1/1 [==============================] - 0s 